# 패키지 설치

In [ ]:
!pip install pandas fiona shapely pyproj rtree

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 16.7 MB 4.4 MB/s 
     |████████████████████████████████| 6.3 MB 37.2 MB/s 
     |████████████████████████████████| 1.0 MB 5.2 MB/s 


In [ ]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 5.1 MB/s 


In [ ]:
import geopandas as gpd
import pandas as pd

# 집계구 데이터 불러오기

In [ ]:
standard_data = gpd.read_file('/content/drive/MyDrive/aaa/집계구.shp')

In [ ]:
standard_data.crs = "epsg:5179"
standard_data = standard_data.to_crs(epsg=5179)

In [ ]:
standard_data.columns

Index(['TOT_REG_CD', 'ADM_NM', 'ADM_CD', 'geometry'], dtype='object')

In [ ]:
standard_data= standard_data.astype({'TOT_REG_CD':'int64'})
standard_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 19153 entries, 0 to 19152
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   TOT_REG_CD  19153 non-null  int64   
 1   ADM_NM      19153 non-null  object  
 2   ADM_CD      19153 non-null  object  
 3   geometry    19153 non-null  geometry
dtypes: geometry(1), int64(1), object(2)
memory usage: 598.7+ KB


# QGIS 데이터 불러오기(학교, 버스)

In [ ]:
life_data = gpd.read_file('/content/drive/MyDrive/청년인재팀플/data/QGIS/사고지점_독립변수 속성테이블 결합/12-20년도/1220사고지점_중심점추가.shp')

In [ ]:
life_data.columns

Index(['FID', '총사고건수', '사망자수', '중상자수', '경상자수', '부상신고자', '경도', '위도', 'hospital',
       'bus', 'subway', 'univ', '초중고', 'CarLane100', 'police', '횡단보도', '무단횡단방',
       '어린이보호', '노인보호', 'xcoord', 'ycoord', 'geometry'],
      dtype='object')

In [ ]:
qgis_stand = standard_data.sjoin(life_data, how="inner", predicate='contains')

# 생활인구 데이터 결합 (야간)

In [ ]:
people_data=pd.read_csv('/content/drive/MyDrive/청년인재팀플/data/QGIS_생활인구 야간 평균.csv',encoding='cp949')

In [ ]:
people_data.columns

Index(['TOT_REG_CD', 'ADM_NM', 'ADM_CD', 'xcoord', 'ycoord', '집계구코드', '주간야간',
       '행정동코드', '총생활인구수', '남자 10대 인구수', '남자 20대 인구수', '남자 30대 인구수',
       '남자 40대 인구수', '남자 50대 인구수', '남자 60대 인구수', '남자 70대 이상 인구수', '여자 10대 인구수',
       '여자 20대 인구수', '여자 30대 인구수', '여자 40대 인구수', '여자 50대 인구수', '여자 60대 인구수',
       '여자 70대 이상 인구수', '기준일ID', '월1'],
      dtype='object')

In [ ]:
total = pd.merge(qgis_stand,people_data,how='left',left_on='TOT_REG_CD',right_on='TOT_REG_CD')

In [ ]:
total.columns

Index(['TOT_REG_CD', 'ADM_NM_x', 'ADM_CD_x', 'geometry', 'index_right', 'FID',
       '총사고건수', '사망자수', '중상자수', '경상자수', '부상신고자', '경도', '위도', 'hospital', 'bus',
       'subway', 'univ', '초중고', 'CarLane100', 'police', '횡단보도', '무단횡단방',
       '어린이보호', '노인보호', 'xcoord_x', 'ycoord_x', 'ADM_NM_y', 'ADM_CD_y',
       'xcoord_y', 'ycoord_y', '집계구코드', '주간야간', '행정동코드', '총생활인구수',
       '남자 10대 인구수', '남자 20대 인구수', '남자 30대 인구수', '남자 40대 인구수', '남자 50대 인구수',
       '남자 60대 인구수', '남자 70대 이상 인구수', '여자 10대 인구수', '여자 20대 인구수', '여자 30대 인구수',
       '여자 40대 인구수', '여자 50대 인구수', '여자 60대 인구수', '여자 70대 이상 인구수', '기준일ID',
       '월1'],
      dtype='object')

In [ ]:
night_data = total.drop(['ADM_NM_x', 'ADM_CD_x', 'index_right', 'FID',
       '경도', '위도', 'xcoord_x', 'ycoord_x', 'ADM_NM_y', 'ADM_CD_y',
       'xcoord_y', 'ycoord_y', '집계구코드', '주간야간', '행정동코드','기준일ID', '월1'],axis=1)

In [ ]:
night_data['야간 남자 20-50대'] = night_data['남자 20대 인구수']+night_data['남자 30대 인구수']+night_data['남자 40대 인구수']+night_data['남자 50대 인구수']

In [ ]:
night_data['야간 여자 20-50대'] = night_data['여자 20대 인구수']+night_data['여자 30대 인구수']+night_data['여자 40대 인구수']+night_data['여자 50대 인구수']

In [ ]:
night_data['야간 남자 60대 이상'] = night_data['남자 60대 인구수']+night_data['남자 70대 이상 인구수']
night_data['야간 여자 60대 이상'] = night_data['여자 60대 인구수']+night_data['여자 70대 이상 인구수']

In [ ]:
night_data.rename(columns={'총생활인구수':'야간총생활인구수','남자 10대 인구수':'야간 남자 10대 이하','여자 10대 인구수':'야간 여자 10대 이하'},inplace=True)

In [ ]:
night_data = night_data.drop(['남자 20대 인구수', '남자 30대 인구수', '남자 40대 인구수', '남자 50대 인구수', '남자 60대 인구수',
       '남자 70대 이상 인구수','여자 20대 인구수', '여자 30대 인구수',
       '여자 40대 인구수', '여자 50대 인구수', '여자 60대 인구수', '여자 70대 이상 인구수'],axis=1)

In [ ]:
night_data.columns

Index(['TOT_REG_CD', 'geometry', '총사고건수', '사망자수', '중상자수', '경상자수', '부상신고자',
       'hospital', 'bus', 'subway', 'univ', '초중고', 'CarLane100', 'police',
       '횡단보도', '무단횡단방', '어린이보호', '노인보호', '야간총생활인구수', '야간 남자 10대 이하',
       '야간 여자 10대 이하', '야간 남자 20-50대', '야간 여자 20-50대', '야간 남자 60대 이상',
       '야간 여자 60대 이상'],
      dtype='object')

# 생활인구 데이터 (주간)

In [ ]:
people_data2=pd.read_csv('/content/drive/MyDrive/청년인재팀플/data/QGIS_생활인구 주간 평균.csv',encoding='cp949')

In [ ]:
people_data2['주간 남자 20-50대'] = people_data2['남자 20대 인구수']+people_data2['남자 30대 인구수']+people_data2['남자 40대 인구수']+people_data2['남자 50대 인구수']
people_data2['주간 여자 20-50대'] = people_data2['여자 20대 인구수']+people_data2['여자 30대 인구수']+people_data2['여자 40대 인구수']+people_data2['여자 50대 인구수']
people_data2['주간 남자 60대 이상'] = people_data2['남자 60대 인구수']+people_data2['남자 70대 이상 인구수']
people_data2['주간 여자 60대 이상'] = people_data2['여자 60대 인구수']+people_data2['여자 70대 이상 인구수']
people_data2.rename(columns={'총생활인구수':'주간총생활인구수','남자 10대 인구수':'주간 남자 10대 이하','여자 10대 인구수':'주간 여자 10대 이하'},inplace=True)
people_data2 = people_data2.drop(['남자 20대 인구수', '남자 30대 인구수', '남자 40대 인구수', '남자 50대 인구수', '남자 60대 인구수',
       '남자 70대 이상 인구수','여자 20대 인구수', '여자 30대 인구수',
       '여자 40대 인구수', '여자 50대 인구수', '여자 60대 인구수', '여자 70대 이상 인구수'],axis=1)

In [ ]:
people_data2.columns

Index(['TOT_REG_CD', 'ADM_NM', 'ADM_CD', 'xcoord', 'ycoord', '집계구코드', '주간야간',
       '행정동코드', '주간총생활인구수', '주간 남자 10대 이하', '주간 여자 10대 이하', '기준일ID', '월1',
       '주간 남자 20-50대', '주간 여자 20-50대', '주간 남자 60대 이상', '주간 여자 60대 이상'],
      dtype='object')

In [ ]:
final_data = pd.merge(night_data,people_data2)

In [ ]:
final_data = final_data.drop(['ADM_NM', 'ADM_CD', 'xcoord', 'ycoord','기준일ID', '월1'],axis=1)

In [ ]:
final_data.shape

(553, 35)

In [ ]:
final_da

In [ ]:
final_data.to_csv('/content/drive/MyDrive/청년인재팀플/data/사고다발지_생활인구_시각화데이터.csv')